# ECE NLP

## Overview

## Imports

In [1]:
import numpy as np
import pandas as pd

## Data import

#### Import train and test csv files

In [14]:
train_df = pd.read_csv("Data/dataset_train.csv",sep='\t', index_col=False)
test_df = pd.read_csv("Data/dataset_test_no_labels.csv",sep='\t')


In [15]:
train_df

,index,sentence_1,sentence_2,label
0,0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,neutral
1,1,you know during the season and i guess at at y...,You lose the things to the following level if ...,entailment
2,2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,entailment
3,3,How do you know? All this is their information...,This information belongs to them.,entailment
4,4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,neutral
...,...,...,...,...
392657,392657,"Clearly, California can - and must - do better.",California cannot do any better.,contradiction
392658,392658,It was once regarded as the most beautiful str...,So many of the original buildings had been rep...,neutral
392659,392659,Houseboats are a beautifully preserved traditi...,The tradition of houseboats originated while t...,entailment
392660,392660,Obituaries fondly recalled his on-air debates ...,The obituaries were beautiful and written in k...,neutral


## Data Transformation

## Model Definition

## Model Training/Evaluation

## Model Inference

## Model Exportation